In [7]:
%reload_ext autoreload
%autoreload 3

import torch
from torch import nn
import pytorch_lightning as pl
from src.seq2conn.models.simple_vae import SimpleVAE
from src.seq2conn.data import RandomDataModule

In [60]:

from torch_geometric.utils import to_edge_index
from torch_geometric.nn import GCNConv

n_nodes = 11
node_features_dim = 12
node_embedding_1_dim = 3
node_embedding_2_dim = 2

adj = torch.randint(0,2, (n_nodes, n_nodes)).to_sparse()
node_features = torch.randn(n_nodes, node_features_dim)
edge_index = to_edge_index(adj)[0]

layer_1 = GCNConv(node_features_dim, node_embedding_1_dim)
layer_2 = GCNConv(node_embedding_1_dim, node_embedding_2_dim)

embedding_1 = layer_1(node_features, edge_index)
embedding_2 = layer_2(embedding_1, edge_index)
embedding_2

tensor([[-0.0449,  0.1395],
        [ 0.1222,  0.2979],
        [-0.0469,  0.2163],
        [-0.3334,  0.0234],
        [ 0.0390,  0.2682],
        [-0.1422,  0.1225],
        [-0.1796,  0.0983],
        [-0.1374,  0.1786],
        [-0.0988,  0.1258],
        [-0.2973,  0.0010],
        [-0.2512, -0.0082]], grad_fn=<AddBackward0>)

In [6]:
DGMG(.2, 10, 5)

DGMG(
  (graph_embed): GraphEmbed(
    (node_gating): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Sigmoid()
    )
    (node_to_graph): Linear(in_features=10, out_features=20, bias=True)
  )
  (graph_prop): GraphProp(
    (message_funcs): ModuleList(
      (0-4): 5 x Linear(in_features=21, out_features=20, bias=True)
    )
    (node_update_funcs): ModuleList(
      (0-4): 5 x GRUCell(20, 10)
    )
  )
  (add_node_agent): AddNode(
    (add_node): Linear(in_features=20, out_features=1, bias=True)
    (node_type_embed): Embedding(1, 10)
    (initialize_hv): Linear(in_features=30, out_features=10, bias=True)
  )
  (add_edge_agent): AddEdge(
    (add_edge): Linear(in_features=30, out_features=1, bias=True)
  )
  (choose_dest_agent): ChooseDestAndUpdate(
    (choose_dest): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [ ]:
datamodule = RandomDataModule(batch_size=16, n_timesteps=127, input_dim=18)


model = SimpleVAE(18, 12, 2)

trainer = pl.Trainer(max_epochs=100, log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)